In [218]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import cupy as cp
import spacy
import cudf
import numpy as np
import tqdm 
import pandas as pd
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
STOPWORDS

In [185]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[:5]


In [186]:
len(data_tip)

5

In [187]:
tip_text = data_tip['text']

In [188]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [189]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in doc_spacy]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



In [190]:
tip_text_preprocessed

0                     [avenger, time, lady]
1    [good, desert, tasty, cuban, sandwich]
2                             [open, think]
3                  [decent, fried, chicken]
4      [appetizer, platter, special, lunch]
Name: text, dtype: object

In [191]:
model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)

In [192]:
model3.wv.vectors_vocab 

array([[-0.00021121,  0.00016458,  0.00164532, ..., -0.00238848,
         0.00301433,  0.00144899],
       [ 0.001241  , -0.00192327,  0.00270662, ..., -0.00071154,
         0.00299919,  0.00032285],
       [ 0.00044444,  0.00084044, -0.000596  , ...,  0.00299712,
        -0.00270545,  0.00163906],
       ...,
       [ 0.00225802, -0.00226578,  0.00306893, ..., -0.00150452,
         0.00210656, -0.0011617 ],
       [-0.00181169,  0.00033329, -0.0015857 , ...,  0.00272837,
         0.00204062, -0.00218944],
       [-0.0027276 , -0.00277865,  0.00121383, ..., -0.00218411,
         0.00208561,  0.00298245]], dtype=float32)

In [193]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [225]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience".split()
test_words=list(np.array(test_words).astype(str))

In [228]:
def test_words_andmodel_updating(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
        model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [ ]:
test_words_(model3,test_words)

# Facebook pretrained model

In [ ]:
from gensim.models.fasttext import load_facebook_model

model = load_facebook_model('')

similar_words = model.wv.most_similar(positive=['word'], topn=5)
print(similar_words)


In [ ]:
sims